# Omni Chords - for Cahos

In [ ]:
from collections import Counter
from itertools import accumulate
from math import log2

## Generate Omni-Chords

### Entropy calculations

Using Shannon entropy. Also using "sub-sequence entropy" and "sequence entropy":
For "sub-sequence entropy" we take length n sub sequences of a list and treat them as symbols,
and calculate the entropy of that. Sequence entropy is just the sum of sub-sequence entropies of all lengths

```python
sequence = [2, 1, 2, 2, 1]
entropy = calculate_entropy(sequence)
# 2-length sub-sequence entropy:
subseq_entropy = calculate_entropy([(2, 1), (1, 2), (2, 2), (2, 1)])
```

In [ ]:
def calculate_entropy(sequence):
    # Count the frequency of each element in the sequence
    frequency = Counter(sequence)
    total_count = len(sequence)

    # Calculate the entropy
    entropy = 0
    for count in frequency.values():
        probability = count / total_count
        entropy -= probability * log2(probability)

    return entropy

def calculate_subseq_entropy(sequence, subsequence_length):
    sub_seqs = []
    for idx in range(len(sequence)-subsequence_length+1):
        sub_seqs.append(tuple(sequence[idx:idx+subsequence_length]))
    return calculate_entropy(sub_seqs)

def calculate_sequence_entropy(sequence):
    return sum(calculate_subseq_entropy(sequence, l) for l in range(len(sequence)+1))

The Scale (more aptly chord) has a few properties of interest:

- `sequence_entropy`
- `span`: distance between the lowest and the highest note in chord
- `inervals`: all interval types contained in the chord

In [ ]:
class Scale:
    def __init__(self, deltas):
        self.deltas = deltas

    def __repr__(self):
        return f"{self.deltas}"

    def __str__(self):
        return f"entropy: {self.sequence_entropy:.2f}, span: {self.span}, deltas: {self.deltas}"

    @property
    def pitch_classes(self):
        return [a % 12 for a in accumulate(self.deltas, initial=0)]

    def transposed(self, start=0):
        return [a % 12 for a in accumulate(self.deltas, initial=start)]

    def realization(self, start=0):
        return [a for a in accumulate(self.deltas, initial=start)]

    @property
    def span(self):
        return sum(self.deltas)

    @property
    def intervals(self):
        return list(set(self.deltas))

    @property
    def n_intervals(self):
        return len(set(self.deltas))

    @property
    def entropy(self):
        return calculate_entropy(self.deltas)

    @property
    def transition_entropy(self):
        return calculate_subseq_entropy(self.deltas, 2)

    @property
    def sequence_entropy(self):
        return calculate_sequence_entropy(self.deltas)

### Generating omni-chords subject to some constraints

There are *a lot* of possible omni-chords. So, some constraints are applied during generation
to enable early termination of the search.

- `allowed_intervals`: List of intervals that an omni-chord can contain
- `disallowed_subsequences`: Mostly to exclude chromatic clusters. No consequtive minor deconds, etc.
- `disallowed_beginnings`: Usually don't want narrow intervals between the bass and the next note.
- `max_span`: Maximum distance between the lowest and higihest note for an interval.

In [ ]:
def contains_subsequence(main_list, sub_list):
    for idx in range(len(main_list) - len(sub_list) + 1):
        if main_list[idx: idx + len(sub_list)] == sub_list:
            return True
    return False

def get_scales(allowed_intervals, disallowed_subsequences=[], disallowed_beginnings=[], max_span=88):
    result = []
    get_scales_recursive(allowed_intervals, disallowed_subsequences, disallowed_beginnings, max_span, result)

    return result

def get_scales_recursive(allowed_intervals, disallowed_subsequences, disallowed_beginnings, max_span, deltas_accumulator,
                         pitch_classes=[0], deltas=[], idx=0):
    if idx == 11:
        deltas_accumulator.append(deltas)
        # normal exit
        return
    for next_interval in allowed_intervals:
        # early exits
        if (idx == 0) and (next_interval in disallowed_beginnings):
            continue
        if sum(deltas + [next_interval]) > max_span:
            continue
        if any(contains_subsequence(deltas + [next_interval], dis_seq) for dis_seq in disallowed_subsequences):
            continue
        next_note = (pitch_classes[-1] + next_interval) % 12
        if next_note not in pitch_classes:
            get_scales_recursive(allowed_intervals, disallowed_subsequences, disallowed_beginnings, max_span, deltas_accumulator,
                      pitch_classes + [next_note], deltas + [next_interval], idx + 1)

In [ ]:
dis_subseq = [
    [1, 2], [2, 1], [3, 1],
    [3, 4], [4, 3],
    [6, 1], [1, 6], [5, 1], [1, 5],
    [1, 1], [2, 2],  # [3, 3], [4, 4],
    [5, 5, 5], [6, 6, 6], [7, 7, 7],
]

# 80 is the maximum span we can get with the Cahos ensemble
scales = [Scale(s) for s in get_scales(
    allowed_intervals=[1, 2, 3, 4, 5, 6, 7],
    disallowed_subsequences=dis_subseq,
    disallowed_beginnings=[],
    max_span=80-12
)]

print(f"number of scales satisfying given constraints: {len(scales)}")

### Further filtering and sorting

Can filter and/or sort by exact number of intervals, entropy, sequence entropy, span...

In [ ]:
n_intervals = 3
selection = sorted([s for s in scales if s.n_intervals == n_intervals], key=lambda x: (x.sequence_entropy, x.span), reverse=True)
show_n = 5
print(f"first {min(show_n, len(selection))} of {len(selection)} scales remaining after filtering")
for s in selection[:show_n]:
    print(s)
print("...")

### Voice leading opportunities

- Number of shared notes between two chords.
- Number of simple swaps among moving notes (ex. 1 -> 0 and 0 -> 1)
- Bass motion

In [ ]:
def n_swaps(xs, ys):
    none_matches = [(a, b) for a, b in zip (xs, ys) if a != b]
    swaps = []
    for item in none_matches:
        if tuple(item[::-1]) in none_matches:
            swaps.append(item)
    
    return len(swaps)

def mark_changes(first, second):
    return [int(a != b) for a, b in zip(first, second)]

### Notes

Create a `VoiceLeading` class that will contain and calculate voice leading information such as:

- a binary array showing changed voices
- sequence entropy of changes indicated by that array
- list of common notes
- list of changed notes
- numbers of common notes, changed notes, swaps
- change in span
- chord a and chord b
- numbers of up/down motions (to find voice leadings with more contrary motion)
- detect voice crossings (not corssings but ex. a b c going to b c d)
    - a changed note (not pitch class) shouldn't be in the previous chord.
- max jump distance of changing voices

Also define a `get_voice_leading(chord_a, `list_of_chords`, n_shared_notes, min_bass_distance=0) -> List[VoiceLeading]` function. list_of_chords doesn't have to be from the same family as `chord_a`.

In [ ]:
note_names = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']

def midi_to_note_name(midi_number):
    octave = (midi_number // 12) - 1
    note_index = midi_number % 12
    return f"{note_names[note_index]}{octave}"

In [ ]:
class VoiceLeading:
    def __init__(self, chord_a: Scale, base_a: int, chord_b: Scale, base_b: int):
        self.chord_a = chord_a
        self.base_a = base_a
        self.chord_b = chord_b
        self.base_b = base_b

    @property
    def real_a(self):
        return self.chord_a.realization(self.base_a)

    @property
    def real_b(self):
        return self.chord_b.realization(self.base_b)

    @property
    def real_a_names(self):
        return [midi_to_note_name(x) for x in self.chord_a.realization(self.base_a)]

    @property
    def real_b_names(self):
         return [midi_to_note_name(x) for x in self.chord_b.realization(self.base_b)]
    
    def onehot_changed_voices(self):
        return [int(a != b) for a, b in zip(self.real_a, self.real_b)]

    def sequence_entropy_of_changes(self):
        return calculate_sequence_entropy(self.onehot_changed_voices())

    def common_notes(self):
        return [a for a, b in zip(self.real_a, self.real_b) if a == b]

    def changed_notes(self):
        return [(a, b) for a, b in zip(self.real_a, self.real_b) if a != b]

    def swaps(self):
        none_matches = [(a, b) for a, b in zip (self.real_a, self.real_b) if a != b]
        swaps = []
        for item in none_matches:
            if tuple(item[::-1]) in none_matches:
                swaps.append(item)
        
        return swaps

    def n_common_notes(self):
        return len(self.common_notes())

    def n_changed_notes(self):
        return len(self.changed_notes())

    def n_swaps(self):
        return len(self.swaps())

    def change_in_span(self):
        return self.chord_b.span - self.chord_a.span

    def n_upward_motion(self):
        n = 0
        for a, b in self.changed_notes():
            if b > a: n += 1
        return n

    def n_downward_motion(self):
        n = 0
        for a, b in self.changed_notes():
            if a > b: n += 1
        return n

    def motion_balance(self):
        if self.n_downward_motion() == 0: return 0.0
        ratio = self.n_upward_motion() / self.n_downward_motion()
        if ratio > 1.0: ratio = 1 / ratio
        return ratio

    def max_step_size(self):
        return max([abs(a-b) for a, b in self.changed_notes()])

    def n_pseudo_changes(self):
        n = 0
        for a, b in self.changed_notes():
            if b in self.real_a:
                n += 1
        return n

    def __repr__(self):
        return f"seq entropy: {self.chord_a.sequence_entropy:.2f} span: {self.chord_a.span} | {self.real_a_names}\n" \
               f"seq entropy: {self.chord_b.sequence_entropy:.2f} span: {self.chord_b.span} | {self.real_b_names}\n" \
               f"{self.n_common_notes()} common | {self.n_changed_notes()} changes | {self.n_swaps()} swaps\n" \
               f"changed places: {self.onehot_changed_voices()}\n" \
               f"sequence entropy of changes: {self.sequence_entropy_of_changes():.2f}\n" \
               f"max step size: {self.max_step_size()}\n"

In [ ]:
min_bass_distance = 2
n_shared_notes = 5

for ss in selection:
    for st in selection:
        for i in range(12):
            shifted = [(pc + i) % 12 for pc in st.pitch_classes]
            matches = [a for a, b in zip(ss.pitch_classes, shifted) if a == b]
            d1 = (shifted[0] - ss.pitch_classes[0]) % 12
            d2 = (ss.pitch_classes[0] - shifted[0]) % 12
            if (len(matches) == n_shared_notes) and (d1 > min_bass_distance) and (d2 > min_bass_distance):
                print(f"span: {ss.span}", ss.pitch_classes)
                print(f"span: {st.span}", shifted)
                print(f"common voices: {matches}")
                print(f"{len(matches)} common | {12 - len(matches)} changes | {n_swaps(shifted, ss.pitch_classes)} swaps")
                print(f"marked changes: {mark_changes(ss.pitch_classes, shifted)}")
                print(f"sequence entropy of changes: {calculate_sequence_entropy(mark_changes(ss.pitch_classes, shifted)):.2f}")
                print()

## Generate Composition Material

### Epigraph

- Nothing changes because everything is already there (The history, empires, art, architecture, cradle of civilizations).
- Everything changes because chaos is the fountain from which cosmos emerges (Yet still creative, leading, etc.).

### Notes

Some properties of interest are:

scale:

- span
- sequence entropy
- number of intervals / allowed intervals

voice leading:

- sequence entropy of changes
- list of common notes
- list of tuples of changed notes
- numbers of common notes, changed notes, swaps
- change in span

In [ ]:
n_shared_notes = 4

b1 = 43
b2 = 40
voice_leading_opportunities = []
for ss in selection:
    for st in selection:
        shifted = st.realization(b2)
        matches = [a for a, b in zip(ss.realization(b1), shifted) if a == b]
        if shifted[-1] > 108 or ss.realization(b1)[-1] > 108:
            continue
        if (len(matches) == n_shared_notes) and ((st.span - ss.span) >= 12):  # and (calculate_sequence_entropy(mark_changes(ss.realization(b1), shifted)) > 24.11):
            voice_leading_opportunities.append(VoiceLeading(ss, b1, st, b2))
            print(f"span: {ss.span}", [midi_to_note_name(x) for x in ss.realization(b1)])
            print(f"span: {st.span}", [midi_to_note_name(x) for x in shifted])
            print(f"common voices: {[midi_to_note_name(x) for x in matches]}")
            print(f"{len(matches)} common | {12 - len(matches)} changes | {n_swaps(shifted, ss.realization(b1))} swaps")
            print(f"marked changes: {mark_changes(ss.realization(b1), shifted)}")
            print(f"sequence entropy of changes: {calculate_sequence_entropy(mark_changes(ss.realization(b1), shifted)):.2f}")
            print()

In [ ]:
len(voice_leading_opportunities)

In [ ]:
n_shared_notes = 4

b1 = 43
b2 = 40
voice_leading_opportunities = []
for c1 in selection:
    for c2 in selection:
        vl = VoiceLeading(c1, b1, c2, b2)
        if vl.real_a[-1] > 108 or vl.real_b[-1] > 108 or vl.real_a[0] < 28 or vl.real_b[0] < 28 or vl.motion_balance() < 3/4 or vl.n_swaps() > 0:
            continue
        if vl.n_common_notes() == n_shared_notes and vl.change_in_span() >= 12 and vl.n_pseudo_changes() == 0:
            voice_leading_opportunities.append(vl)

In [ ]:
for vl in sorted(voice_leading_opportunities, key=lambda x: (-x.sequence_entropy_of_changes(), x.n_swaps())):
    print(f"{vl.chord_a.sequence_entropy:.2f}", vl.real_a_names, f"{vl.chord_b.sequence_entropy:.2f}", vl.real_b_names, f"{vl.sequence_entropy_of_changes():.2f}", vl.n_swaps())

In [ ]:
n_shared_notes = 5

b1 = 40
b2 = 42
voice_leading_opportunities = []
for c1 in selection:
    for c2 in selection:
        vl = VoiceLeading(c1, b1, c2, b2)
        if max(vl.real_a + vl.real_b) > 108 or min(vl.real_a + vl.real_b) < 28 or vl.motion_balance() < 3/4 or vl.n_swaps() > 0:
            continue
        if vl.n_common_notes() == n_shared_notes and vl.max_step_size() < 3 and vl.n_pseudo_changes() == 0:
            voice_leading_opportunities.append(vl)

In [ ]:
for vl in sorted(voice_leading_opportunities, key=lambda x: (x.change_in_span(), -x.sequence_entropy_of_changes())):
    print(vl)